In [6]:
import StableDiffusion.ModelConverter
from StableDiffusion.DiffusionProcess import DiffusionProcess
device = 'cuda'
idleDevice = 'cpu'
#diffusionDict = StableDiffusion.ModelConverter.load_from_standard_weights(input_file='./models/inkpunk-diffusion-v1.ckpt',\
#                                                            device = 'cuda')
diffusionDict = StableDiffusion.ModelConverter.load_from_standard_weights(input_file='../models/sd15models/v1-5-pruned-emaonly.ckpt',\
                                                            device = 'cuda')
clipWeights=diffusionDict['clip']
diffusionWeights = diffusionDict['diffusion']
vaeEncoderWeights = diffusionDict['encoder']
vaeDecoderWeights = diffusionDict['decoder']



import torch 
import StableDiffusion.VaeEncoder 
import StableDiffusion.VaeDecoder
import StableDiffusion.ClipEncoder
import StableDiffusion.DiffusionProcess
import importlib
importlib.reload(StableDiffusion.VaeEncoder)
importlib.reload(StableDiffusion.VaeDecoder)
importlib.reload(StableDiffusion.ClipEncoder)
importlib.reload(StableDiffusion.DiffusionProcess)
from StableDiffusion.VaeDecoder import VaeDecoder
from StableDiffusion.VaeEncoder import VaeEncoder
from StableDiffusion.ClipEncoder import ClipEncoder
from StableDiffusion.DiffusionProcess import DiffusionProcess
clipEncoder = ClipEncoder().to(device)
vaeEncoder = VaeEncoder().to(device)
vaeDecoder = VaeDecoder().to(device)
diffusionProcess = DiffusionProcess().to(device)
clipEncoder.load_state_dict(clipWeights,strict=True)
vaeEncoder.load_state_dict(vaeEncoderWeights ,strict=True)
vaeDecoder.load_state_dict(vaeDecoderWeights,strict=True)
diffusionProcess.load_state_dict(diffusionWeights,strict=True)
clipName = clipEncoder.__class__.__name__
print(clipName)




ClipEncoder


In [8]:
clipEncoder = ClipEncoder().to(device)
clipEncoderLora = ClipEncoder().to(device)
clipEncoder.load_state_dict(clipWeights,strict=True)
clipEncoderLora.load_state_dict(clipWeights, strict=True)


<All keys matched successfully>

In [11]:

import torch.nn as nn
import torch
import StableDiffusion.LoraLayer
import importlib
importlib.reload(StableDiffusion.LoraLayer)
from StableDiffusion.LoraLayer import LoraLayer

def injectLora(model,filterTuple=('in_proj','out_proj')):
    for name,layer in model.named_modules():        
        filterName =filterTuple
        if name.endswith(filterName) and isinstance(layer,nn.Linear):            
            partNames = name.split('.')
            preName = '.'.join(partNames[:-1])
            layerName = partNames[-1]
            print(f'preName = {preName}')
            if preName =='':
                parent = model
            else:
                parent = model.get_submodule(preName)    
            injectedLoraLayer = LoraLayer(layer,rank =8,alpha=16)
            setattr(parent,layerName,injectedLoraLayer)
    for name,params in model.named_parameters():
        print(f' {name} {params.size()}')




In [15]:
'lora' in 'loraA'

True

In [13]:
injectLora(clipEncoderLora,('in_proj','out_proj'))
clipLoraDict = dict(clipEncoderLora.named_parameters())
clipDict = dict(clipEncoder.named_parameters())

clipLoraDict['layers.0.attention.in_proj.originLinear.weight']

 embedding.position_embedding torch.Size([77, 768])
 embedding.token_embedding.weight torch.Size([49408, 768])
 layers.0.layernorm_1.weight torch.Size([768])
 layers.0.layernorm_1.bias torch.Size([768])
 layers.0.attention.in_proj.loraA torch.Size([768, 8])
 layers.0.attention.in_proj.loraB torch.Size([8, 2304])
 layers.0.attention.in_proj.originLinear.weight torch.Size([2304, 768])
 layers.0.attention.in_proj.originLinear.bias torch.Size([2304])
 layers.0.attention.out_proj.loraA torch.Size([768, 8])
 layers.0.attention.out_proj.loraB torch.Size([8, 768])
 layers.0.attention.out_proj.originLinear.weight torch.Size([768, 768])
 layers.0.attention.out_proj.originLinear.bias torch.Size([768])
 layers.0.layernorm_2.weight torch.Size([768])
 layers.0.layernorm_2.bias torch.Size([768])
 layers.0.linear_1.weight torch.Size([3072, 768])
 layers.0.linear_1.bias torch.Size([3072])
 layers.0.linear_2.weight torch.Size([768, 3072])
 layers.0.linear_2.bias torch.Size([768])
 layers.1.layernorm_1.w

Parameter containing:
tensor([[ 0.0139,  0.0147, -0.0089,  ..., -0.0349, -0.0042, -0.0188],
        [-0.0586, -0.0059, -0.0179,  ...,  0.0012, -0.0068,  0.0254],
        [-0.0211, -0.0321,  0.0308,  ..., -0.0189,  0.0091,  0.0066],
        ...,
        [-0.0217, -0.0089, -0.0143,  ..., -0.0153,  0.0053,  0.0016],
        [-0.0086, -0.0083, -0.0049,  ...,  0.0208, -0.0048, -0.0041],
        [-0.0087, -0.0024,  0.0105,  ..., -0.0037, -0.0148,  0.0030]],
       device='cuda:0', requires_grad=True)

In [14]:
clipDict['layers.0.attention.in_proj.weight']

Parameter containing:
tensor([[ 0.0139,  0.0147, -0.0089,  ..., -0.0349, -0.0042, -0.0188],
        [-0.0586, -0.0059, -0.0179,  ...,  0.0012, -0.0068,  0.0254],
        [-0.0211, -0.0321,  0.0308,  ..., -0.0189,  0.0091,  0.0066],
        ...,
        [-0.0217, -0.0089, -0.0143,  ..., -0.0153,  0.0053,  0.0016],
        [-0.0086, -0.0083, -0.0049,  ...,  0.0208, -0.0048, -0.0041],
        [-0.0087, -0.0024,  0.0105,  ..., -0.0037, -0.0148,  0.0030]],
       device='cuda:0', requires_grad=True)

In [2]:
import StableDiffusion.LoraLayer
import importlib
importlib.reload(StableDiffusion.LoraLayer)
from StableDiffusion.LoraLayer import LoraLayer



clipLayersDict = dict(clipEncoder.named_modules())
rawLinear = clipLayersDict['layers.11.attention.out_proj']



In [36]:
lora1 = LoraLayer(rawLinear,rank = 8,alpha =16)
for name, param in lora1.named_parameters():
    print("-" * 30)
    print(f"Parameter Name: {name}")
    print(f"Shape: {param.shape}")
    print(f"Requires Grad: {param.requires_grad}")
    # print(param.data) # Uncomment this if you want to see the actual numbers

lora1.to(device)


------------------------------
Parameter Name: loraA
Shape: torch.Size([768, 8])
Requires Grad: True
------------------------------
Parameter Name: loraB
Shape: torch.Size([8, 768])
Requires Grad: True
------------------------------
Parameter Name: originLinear.weight
Shape: torch.Size([768, 768])
Requires Grad: False
------------------------------
Parameter Name: originLinear.bias
Shape: torch.Size([768])
Requires Grad: False


LoraLayer(
  (originLinear): Linear(in_features=768, out_features=768, bias=True)
)

In [17]:
print(lora1.alpha)
print(lora1.rank)
print(lora1.scale)
print(f'lora1.loraA  {lora1.loraA} {lora1.loraA.shape}')
print(f'lora1.loraB  {lora1.loraB}  {lora1.loraB.shape}')
      

16
8
2.0
lora1.loraA  Parameter containing:
tensor([[ 0.0962,  0.0491, -0.1887,  ..., -0.3529, -0.0487,  0.2995],
        [ 0.0673,  0.0200,  0.0351,  ...,  0.5498,  0.2075, -0.2122],
        [ 0.4916, -0.4762,  0.4006,  ..., -0.2131, -0.0747,  0.2232],
        ...,
        [-0.1283,  0.1568, -0.2501,  ...,  0.0059,  0.2656, -0.1761],
        [-0.0360,  0.3712, -0.1295,  ..., -0.2958, -0.2810,  0.1501],
        [ 0.2324, -0.2738,  0.1563,  ...,  0.4901, -0.2719, -0.2658]],
       device='cuda:0', requires_grad=True) torch.Size([768, 8])
lora1.loraB  Parameter containing:
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0', requires_grad=True)  torch.Size([8, 768])


In [46]:
for name, module in clipEncoder.named_modules():
    # 'name' is the string path (e.g., 'layers.0.self_attn.q_proj')
    # 'module' is the actual layer object
    print(f"Name: {name} | Type: {type(module)}")

Name:  | Type: <class 'StableDiffusion.ClipEncoder.ClipEncoder'>
Name: embedding | Type: <class 'StableDiffusion.ClipEncoder.ClipEmbedding'>
Name: embedding.token_embedding | Type: <class 'torch.nn.modules.sparse.Embedding'>
Name: layers | Type: <class 'torch.nn.modules.container.ModuleList'>
Name: layers.0 | Type: <class 'StableDiffusion.ClipEncoder.ClipEncoderLayer'>
Name: layers.0.layernorm_1 | Type: <class 'torch.nn.modules.normalization.LayerNorm'>
Name: layers.0.attention | Type: <class 'StableDiffusion.Attention.MHSelfAttention'>
Name: layers.0.attention.in_proj | Type: <class 'StableDiffusion.LoraLayer.LoraLayer'>
Name: layers.0.attention.in_proj.originLinear | Type: <class 'torch.nn.modules.linear.Linear'>
Name: layers.0.attention.out_proj | Type: <class 'StableDiffusion.LoraLayer.LoraLayer'>
Name: layers.0.attention.out_proj.originLinear | Type: <class 'torch.nn.modules.linear.Linear'>
Name: layers.0.layernorm_2 | Type: <class 'torch.nn.modules.normalization.LayerNorm'>
Name:

In [49]:
import torch
import torch.nn as nn
model = clipEncoder
for name,layer in model.named_modules():
    nameSplits=name.split('.')
    filterName = ('in_proj','out_proj')
    if name.endswith(filterName) and isinstance(layer,nn.Linear):
        print(name)
        partNames = name.split('.')
        preName = '.'.join(partNames[:-1])
        layerName = partNames[-1]
        print(f'preName = {preName}')
        if preName =='':
            parent = model
        else:
            parent = model.get_submodule(preName)
            print(f'parent = {parent} {type(parent)}')
    
        injectedLoraLayer = LoraLayer(layer,rank =8,alpha=16)
        setattr(parent,layerName,injectedLoraLayer)
            


In [50]:
def count_trainable_parameters(model):
    # Total trainable params
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    
    # Non-trainable (frozen) params
    frozen_params = sum(p.numel() for p in model.parameters() if not p.requires_grad)
    
    print(f"Trainable (LoRA): {trainable_params:,}")
    print(f"Frozen (Original): {frozen_params:,}")
    print(f"Percentage: {100 * trainable_params / (trainable_params + frozen_params):.4f}%")

count_trainable_parameters(clipEncoder)

Trainable (LoRA): 442,368
Frozen (Original): 123,060,480
Percentage: 0.3582%


In [51]:
for name,params in model.named_parameters():
    print(f' {name} {params.size()}')

 embedding.position_embedding torch.Size([77, 768])
 embedding.token_embedding.weight torch.Size([49408, 768])
 layers.0.layernorm_1.weight torch.Size([768])
 layers.0.layernorm_1.bias torch.Size([768])
 layers.0.attention.in_proj.loraA torch.Size([768, 8])
 layers.0.attention.in_proj.loraB torch.Size([8, 2304])
 layers.0.attention.in_proj.originLinear.weight torch.Size([2304, 768])
 layers.0.attention.in_proj.originLinear.bias torch.Size([2304])
 layers.0.attention.out_proj.loraA torch.Size([768, 8])
 layers.0.attention.out_proj.loraB torch.Size([8, 768])
 layers.0.attention.out_proj.originLinear.weight torch.Size([768, 768])
 layers.0.attention.out_proj.originLinear.bias torch.Size([768])
 layers.0.layernorm_2.weight torch.Size([768])
 layers.0.layernorm_2.bias torch.Size([768])
 layers.0.linear_1.weight torch.Size([3072, 768])
 layers.0.linear_1.bias torch.Size([3072])
 layers.0.linear_2.weight torch.Size([768, 3072])
 layers.0.linear_2.bias torch.Size([768])
 layers.1.layernorm_1.w

In [45]:
for name,layer in model.named_modules():
    print(f' {name} {layer}')




  ClipEncoder(
  (embedding): ClipEmbedding(
    (token_embedding): Embedding(49408, 768)
  )
  (layers): ModuleList(
    (0-11): 12 x ClipEncoderLayer(
      (layernorm_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attention): MHSelfAttention(
        (in_proj): LoraLayer(
          (originLinear): Linear(in_features=768, out_features=2304, bias=True)
        )
        (out_proj): LoraLayer(
          (originLinear): Linear(in_features=768, out_features=768, bias=True)
        )
      )
      (layernorm_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (linear_1): Linear(in_features=768, out_features=3072, bias=True)
      (linear_2): Linear(in_features=3072, out_features=768, bias=True)
    )
  )
  (layernorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
)
 embedding ClipEmbedding(
  (token_embedding): Embedding(49408, 768)
)
 embedding.token_embedding Embedding(49408, 768)
 layers ModuleList(
  (0-11): 12 x ClipEncoderLayer(
    (layernorm_

In [ ]:
'''   
    filterName = 'k_proj'
    if name.endswith(filterName):
        print(name)
        
    filterName = 'v_proj'
    if name.endswith(filterName):
        print(name)
    
    filterName = 'q_proj'
    if name.endswith(filterName):
        print(name)
    
'''    

    #print(name_cols)

In [19]:
name = clipEncoder
name_cols=name.split('.')

# 逐层下探到linear归属的module
children=name_cols[:-1]
cur_layer=model 
for child in children:
    cur_layer=getattr(cur_layer,child)

#print(layer==getattr(cur_layer,name_cols[-1]))
#lora_layer=LoraLayer(layer,layer.in_features,layer.out_features,LORA_R,LORA_ALPHA)
#setattr(cur_layer,name_cols[-1],lora_layer)

AttributeError: 'ClipEncoder' object has no attribute 'split'